In [2]:
%pip install ultralytics
%pip install cvzone
%pip install filterpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from ultralytics import YOLO
import cv2, cvzone, torch
import math
from sort import *

video_source = "cars.mp4"
cap = cv2.VideoCapture(video_source)

cuda_devices = torch.cuda.device_count()
if cuda_devices > 0:
    cuda_devices = [i for i in range(cuda_devices)]
else:
    cuda_devices = "cpu"

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

model = YOLO("Yolo-Weights/yolov8l.pt")

# Save output video to file
out = cv2.VideoWriter('Tracking_cars.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Tracker
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits = [400, 297, 673, 297]
totalCount = []
frame_number = 0  # To keep track of frames

while True:
    success, img = cap.read()
    
    if not success:
        print("Failed to read video or video ended")
        break

    # Process the frame using YOLO model
    results = model(img, stream=True, device=cuda_devices)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Filter only cars, trucks, or buses
            if (currentClass == "car" or currentClass == "truck" or currentClass == "bus") and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    resultsTracker = tracker.update(detections)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)

        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

    # Write the processed frame to output video
    out.write(img)

    # Optional: Save the frame as an image for debugging purposes
    # Uncomment the following line if you want to save frames for inspection
    # cv2.imwrite(f'frame_{frame_number}.jpg', img)

    frame_number += 1

    # Use cv2.waitKey() if needed, but since we're not displaying the image, it's not necessary

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 378.0ms
Speed: 2.2ms preprocess, 378.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 423.4ms
Speed: 2.1ms preprocess, 423.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 16 cars, 1 truck, 1 traffic light, 1 stop sign, 566.6ms
Speed: 2.0ms preprocess, 566.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 16 cars, 1 traffic light, 1 stop sign, 315.8ms
Speed: 2.0ms preprocess, 315.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 2 trucks, 1 traffic light, 334.0ms
Speed: 3.0ms preprocess, 334.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 1 traffic light, 332.4ms
Speed: 1.0ms preprocess, 332.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 64